In [3]:
# data processing
import numpy as np
import pandas as pd
# pyplot
from matplotlib import pyplot as plt
# seaborn
import seaborn as sns
# ipywidgets
from ipywidgets import interact
# bokeh
from bokeh import plotting as bk
from bokeh.io import output_notebook, push_notebook
from bokeh.models import (
    ColumnDataSource, 
    Span, 
    HoverTool, 
    Slider, 
    CustomJS, 
    Label, 
    Text, 
    Select, 
    Range1d
)
from bokeh.plotting import figure, output_file, show, curdoc
from bokeh.layouts import column, row
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.events import Tap
#utils
import os
from IPython.display import clear_output
output_notebook()

Loading BokehJS ...

In [4]:
# Compound utils
from compound import (
    get_compound_return,
    simulate_compound_return,
    define_scenario,
    plot_scenario_bokeh,
    build_dataframe,
    MONTHS_IN_YEAR,
    INT_TO_TERM_NAME
)

1. Server-side solution to call Python _callback in Bokeh

https://stackoverflow.com/questions/61695005/updating-a-graph-using-a-slider-with-python-and-bokeh

1. Grid lines

https://stackoverflow.com/questions/36244811/how-do-i-remove-grid-lines-from-a-bokeh-plot

1. Visual styles

https://docs.bokeh.org/en/2.4.1/docs/user_guide/styling.html

1. Update button text by JS code

https://discourse.bokeh.org/t/change-button-label-from-javascript/3549

1. Sum by reduce() in JS

https://stackoverflow.com/questions/1230233/how-to-find-the-sum-of-an-array-of-numbers

1. Mouse event and _callback

https://stackoverflow.com/questions/62235067/how-to-get-mouse-position-with-bokeh-server

1. Label() font size

https://stackoverflow.com/questions/46379968/change-the-text-size-of-bokeh-label-annotations

1. Label visible and not visible.

https://discourse.bokeh.org/t/how-to-clear-old-labels-from-a-plot/7293

1. Events in selects

https://github.com/bokeh/bokeh/issues/9993

1. Slider with disabled property added in 2016.

https://github.com/bokeh/bokeh/issues/1376

1. Select widget is similar to Dropdown widget, but allows both access of property 'value' and on_change(event) attribute

https://docs.bokeh.org/en/latest/docs/reference/models/widgets/inputs.html#bokeh.models.Select

1. To update a title from a figure plot, you have to call 'title', that retrieves a 'Title' instance which contains a property 'text', then call 'text'

https://discourse.bokeh.org/t/how-to-update-figure-title/9179

In [5]:
def make_page(doc, w=600, h=500):
    def _callback_slider(attr, old, new):
        _update_plots()
    
    def _update_plots():
        # Validate and set tax
        if select_country_tax.value != "manual rate":
            slider_tax_rate.disabled = True
            tax = select_country_tax.value
        else:
            slider_tax_rate.disabled = False
            tax = slider_tax_rate.value
        # Calculate the total balance and earnings using the updated function
        total_balance, info = simulate_compound_return(
            principal=slider_initial_amount.value,
            annual_roi=slider_annual_roi.value,
            yield_frequency=slider_yield_frequency.value,
            annual_contribution=slider_annual_contribution.value,
            inc_contribution_rate=slider_contribution_inc.value,
            investment_duration=slider_duration_years.value,
            retirement_at=slider_extraction_start.value,
            monthly_retirement_income=slider_monthly_extraction.value,
            inflation_rate=slider_inflation_rate.value,
            tax=tax,
            return_series=True
        )
        source_e.data = dict(term=np.arange(len(info["net_earnings"])),
                             earning=info["net_earnings"]
                            )
        source_b.data = dict(month=np.arange(len(info["balances"])),
                             year=np.arange(len(info["balances"])) // 12,
                             balance=info["balances"]
                            )
        # Update earning label to zero
        label_earnings.text = f"Total earnings at term {0} : € {0}"
        # Titles
        new_term_name = INT_TO_TERM_NAME.get(slider_yield_frequency.value, '').capitalize()
        plot_earnings.title.text = f"{new_term_name} evolution of earnings"
        
    _, info = simulate_compound_return(
        principal=10_000,
        annual_roi=0.06,
        yield_frequency=12,
        annual_contribution=1200,
        inc_contribution_rate=0.01,
        investment_duration=60,
        retirement_at=30,
        monthly_retirement_income=500,
        inflation_rate=0.02,
        tax=0.2,
        return_series=True
    )
    earnings = info["net_earnings"]
    balances = info["balances"]

    plot_earnings = bk.figure(width=w,
                              height=h,
                              title=f"{INT_TO_TERM_NAME[12].capitalize()} evolution of earnings",
                              #tools='hover'
                              )
    # Annotations
    plot_earnings.xaxis.axis_label = "term"
    plot_earnings.yaxis.axis_label = "earning (€)"
    # Grids
    plot_earnings.xgrid.grid_line_color = None
    # Background color
    plot_earnings.background_fill_color = "cyan"
    plot_earnings.background_fill_alpha = 0.1
    # Border color
    plot_earnings.border_fill_color = "gray"
    plot_earnings.border_fill_alpha = 0.2
    
    ## Earnings plot
    source_e = ColumnDataSource(
        data=dict(term=np.arange(len(earnings)),
                  earning=earnings
                 )
    )

    curve_e = plot_earnings.line(x='term',
                                 y='earning',
                                 source=source_e)

    # initial-earning baseline
    hline_e = Span(location=earnings[0],
                   dimension='width',
                   line_color='red',
                   line_width=1,
                   line_dash='dashed')

    #plot.renderers.extend([curve, hline])

    # hover tools for earning curve
    hover_e = HoverTool(
        tooltips=[
            ('term', '@term'),
            ('earning-yield', '€ @earning{%0.2f}'), # use @{ } for field names with spaces
        ],

        formatters={
            '@term': 'printf', # use default 'numeral' formatter for other fields
            '@earning': 'printf',   # use 'printf' formatter for '@{adj close}' field
        },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
    )

    ## Balances plot
    plot_balances = bk.figure(width=w,
                              height=h,
                              title='Annual evolution of balance',
                              #tools='hover'
                             )
    # Annotations
    plot_balances.xaxis.axis_label = "year"
    plot_balances.yaxis.axis_label = "balance (€)"
    # Grids
    plot_balances.xgrid.grid_line_color = None
    # Background
    plot_balances.background_fill_color = "cyan"
    plot_balances.background_fill_alpha = 0.1
    
    # Border color
    plot_balances.border_fill_color = "gray"
    plot_balances.border_fill_alpha = 0.2
    
    source_b = ColumnDataSource(
        data=dict(month=np.arange(len(balances)),
                  year=np.arange(len(balances)) // 12,
                  balance=balances
                  )

        )
    curve_b = plot_balances.line(x='year',
                                 y='balance',
                                 source=source_b)

    # zero-balance baseline
    hline_b = Span(location=earnings[0],
                   dimension='width',
                   line_color='red',
                   line_width=1,
                   line_dash='dashed')
    
    # Sliders
    slider_initial_amount = Slider(start=0.0, 
                                   end=100_000.0, 
                                   value=10_000.0, 
                                   step=200.0, 
                                   title="Initial amount"
                                  )
    slider_initial_amount.on_change("value", _callback_slider)
    slider_annual_roi = Slider(start=0.0, 
                               end=1.0, 
                               value=0.06, 
                               step=0.01, 
                               title="Roi"
                              )
    slider_annual_roi.on_change("value", _callback_slider)
    slider_yield_frequency = Slider(start=1, 
                                    end=12, 
                                    value=12, 
                                    step=1, 
                                    title="Yield frequency"
                                   )
    slider_yield_frequency.on_change("value", _callback_slider)
    slider_annual_contribution = Slider(start=0.0, 
                                        end=10_000.0, 
                                        value=1200.0, 
                                        step=50.0, 
                                        title="Annual contribution"
                                       )
    slider_annual_contribution.on_change("value", _callback_slider)
    slider_contribution_inc = Slider(start=0.0, 
                                     end=0.1, 
                                     value=0.01, 
                                     step=0.01, 
                                     title="Contribution increment"
                                    )
    slider_contribution_inc.on_change("value", _callback_slider)
    slider_duration_years = Slider(start=1, 
                                   end=100, 
                                   value=60, 
                                   step=1, 
                                   title="Duration years"
                                  )
    slider_duration_years.on_change("value", _callback_slider)
    slider_extraction_start = Slider(start=1, 
                                     end=100, 
                                     value=30, 
                                     step=1, 
                                     title="Year extraction start"
                                    )
    slider_extraction_start.on_change("value", _callback_slider)
    slider_monthly_extraction = Slider(start=0.0, 
                                       end=10_000.0, 
                                       value=1_000.0, 
                                       step=50.0, 
                                       title="Monthly extraction"
                                      )
    slider_monthly_extraction.on_change("value", _callback_slider)
    slider_inflation_rate = Slider(start=0.0, 
                                   end=0.5, 
                                   value=0.02, 
                                   step=0.005, 
                                   title="Inflation rate"
                                  )
    slider_inflation_rate.on_change("value", _callback_slider)
    slider_tax_rate = Slider(start=0.0, 
                             end=1.0, 
                             value=0.2, 
                             step=0.005, 
                             title="Tax rate"
                            )
    slider_tax_rate.on_change("value", _callback_slider)
    
    # Select widget for tax
    def _callback_select(attr, old, new):
        if new != old:
            _update_plots()
    countries_options = [
        "manual rate", 
        "spain"
    ]
    select_country_tax = Select(title="Tax selection", 
                                value="Manual rate",
                                options=countries_options
                               )
    select_country_tax.on_change("value", _callback_select)
    
    # hover tools for balance curve
    hover_b = HoverTool(
        tooltips=[
            ('month',  '@month (year @year)'),
            ('balance', '€ @balance{%0.2f}'), # use @{ } for field names with spaces
        ],

        formatters={
            '@year': 'printf', # use default 'numeral' formatter for other fields
            '@balance': 'printf',   # use 'printf' formatter for '@{adj close}' field
        },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
    )
    # Label annotations
    def _callback_label_earning(event):
        cursor_x_data = int(event.x)
        accum_earnings_at_term = round(sum(source_e.data['earning'][:cursor_x_data + 1]), 0)
        if cursor_x_data >= 0:
            label_earnings.text = f"Total earnings at term {cursor_x_data} : € {accum_earnings_at_term}"
    
    label_earnings = Label(
        text="Total earnings 0: € 0", 
        x=200, 
        y=400, 
        text_font_size="10pt",
        x_units='screen',
        y_units='screen'
    )
    plot_earnings.on_event("mousemove", _callback_label_earning)
    
    def _callback_label_earning_reference(event):
        cursor_x_data = int(event.x)
        accum_earnings_at_term = round(sum(source_e.data['earning'][:cursor_x_data + 1]), 0)
        if cursor_x_data >= 0:
            label_earnings_reference.text = f"Reference at term {cursor_x_data} : € {accum_earnings_at_term}"
        if not label_earnings_reference.visible:
            label_earnings_reference.visible = True
    
    label_earnings_reference = Label(
        text="Reference 0: € 0", 
        x=200, 
        y=380,
        text_font_size="10pt",
        x_units="screen",
        y_units="screen",
        visible=False
    )
    plot_earnings.on_event(Tap, _callback_label_earning_reference)
    
    # Add layouts and tools
    plot_earnings.add_layout(hline_e)
    plot_earnings.add_layout(label_earnings)
    plot_earnings.add_layout(label_earnings_reference)
    plot_earnings.add_tools(hover_e)
    
    plot_balances.add_layout(hline_b)
    plot_balances.add_tools(hover_b)
    plot_row = row(plot_earnings, plot_balances)
    
    sliders_selects_column = column(slider_initial_amount,
                                      slider_annual_roi,
                                      slider_annual_contribution,
                                      slider_yield_frequency,
                                      slider_duration_years,
                                      slider_contribution_inc,
                                      slider_extraction_start,
                                      slider_inflation_rate,
                                      slider_monthly_extraction,
                                      slider_tax_rate,
                                      select_country_tax,
                                      width=300
                                     )
    doc.add_root(row(plot_row, sliders_selects_column))

In [6]:
try:
    server.stop()
except:
    pass
# creating the application and running the server local, (http://localhost:5000), port 5000 can be changed
apps = {'/': Application(FunctionHandler(make_page))}
server = Server(apps, port=8887)
server.start()

In [7]:
#server.stop()